# Dynamics via acceleration maps

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from analysis import analysis

from importlib import reload

L_box = 50
mu_factor = 6  # conversion to microns
min_factor = 8  # conversion to minutes


In [ ]:
import yaml


def apply_time_filter(df, dt):
    """
    dt : float, in min
    """
    df["ts"] = df["time[hr]"] * 60 // dt
    return df.drop_duplicates(subset=["ts", "rid"], keep="first").reset_index(drop=True)


def _get_mp_type(yfile):
    with open(yfile) as f:
        return yaml.safe_load(f)["substrate"]["kind"]


def _make_title(df):
    tbl = {
        "gamma": r"$\gamma$",
        "R_eq": r"$R_{eq}$",
        "mag_std": r"$\sigma_{MVG}$",
        "add_rate": r"$\tau_{MVG}$",
        "gid": "ID",
    }
    d = dict(df.iloc[0][3:8])
    title = ""
    for key, val in d.items():
        title += tbl[key] + " = " + f"{val}" + "\n"
    return title


def linear_init_pts(xmin, xmax, vmin, vmax, n_pts, s=1, basin_only=False):
    def _around_basin():
        d = 10
        x1 = np.linspace(xmin - d, xmin + d, n_pts)
        x2 = np.linspace(xmax - d, xmax + d, n_pts)
        x = np.append(x1, x2)
        y = s * (vmax - vmin) / (xmax - xmin) * (x - xmin) + vmin
        return np.hstack([x.reshape(-1, 1), y.reshape(-1, 1)])

    if basin_only:
        return _around_basin()

    x = np.linspace(xmin, xmax, n_pts)
    y = s * (vmax - vmin) / (xmax - xmin) * (x - xmin) + vmin
    return np.hstack([x.reshape(-1, 1), y.reshape(-1, 1)])


def F_init_pts(F, bounds):
    y_indx, x_indx = np.where(~np.isnan(F))
    xmin, xmax, vmin, vmax, nbins = bounds

    yy = vmin + y_indx * (vmax - vmin) / nbins
    xx = xmin + x_indx * (xmax - xmin) / nbins
    return np.hstack([xx.reshape(-1, 1), yy.reshape(-1, 1)])


def init_lattice(F, bounds):
    xmin, xmax, vmin, vmax, nbins = bounds
    X, Y = np.meshgrid(
        np.linspace(xmin, xmax, nbins),
        np.linspace(vmin, vmax, nbins),
    )
    non_nans = np.argwhere(~np.isnan(F))
    x, y = X[0][non_nans[:, 1]], Y[:, 0][non_nans[:, 0]]
    return X, Y, np.hstack([x.reshape(-1, 1), y.reshape(-1, 1)])


def get_xv_traj_for_gid(gid):
    df_gid_singleRunLong = pd.read_pickle(
        f"../_server/sim_data/corners_only_2/single_run_long/pkls/fulltake_gid{gid}.pkl"
    )
    df_gid_singleRunLong.x *= mu_factor
    df_gid_singleRunLong.y *= mu_factor

    singleRun_xva = analysis.calc_v_a_from_position(
        df_gid_singleRunLong.x, df_gid_singleRunLong["time[hr]"]
    )
    return singleRun_xva[["x", "v"]].to_numpy()


#### Load grid by grid
- Load `fulltake_gid*.pkl`, which is all of the runs for a config

In [ ]:
from glob import glob

files = glob("../_server/sim_data/corners_only/pkls/fulltake_gid*.pkl")

data = []

for file in files:
    df = pd.read_pickle(file)
    df.x *= mu_factor
    df.y *= mu_factor
    data.append(df)

print(f"Loaded {len(data)} configurations.")
[
    print(
        f"\t - Grid {df.gid.iloc[0]}: {len(df)} total data points | {len(df.rid.unique())} runs | {len(df[df.rid==0])} data points / run"
    )
    for df in data
]
display(data[0])


#### Calculate $\{v, a\}$ and bin $\{x, v, a\}$

In [ ]:
import os

gid = 15
save_path = f"../_server/sim_data/corners_only/resolve_streamplots/12_min_interval/grid {gid}/"
if not os.path.exists(save_path):
    os.makedirs(save_path)

In [ ]:
gid_to_indx = dict((data[k].iloc[0].gid.astype(int), k) for k in range(len(data)))

In [ ]:
df_gid = data[gid_to_indx[gid]]
df_gid = apply_time_filter(df_gid, dt=12)

# compute speed and acc for each run
grid_x_v_a = []
for rid, df_rid in df_gid.groupby("rid"):
    x_v_a = analysis.calc_v_a_from_position(df_rid.x, df_rid["time[hr]"])
    x_v_a[df_gid.iloc[0][5:].index] = df_gid.iloc[0][5:]
    x_v_a["rid"] = int(rid)
    grid_x_v_a.append(x_v_a)

grid_x_v_a = pd.concat(grid_x_v_a)

# with a 112mu mp, this gives bins of dim 3.5mu x 3.5mu
nbins = 32
analysis.get_bin_indices(grid_x_v_a, nbins)
grid_x_v_a


1. Stream lines for every possible lattice site with `broken_streamlines` == `False`

In [ ]:
reload(analysis)

dt = np.mean(np.diff(df_gid["time[hr]"]))  # hr

F, F_std, sigma = analysis.calc_F_sigma(grid_x_v_a, dt, nbins, min_pts=5)

bounds = grid_x_v_a.agg(["min", "max"])
xmin, xmax = bounds["x"]
vmin, vmax = bounds["v"]

plot_title = _make_title(grid_x_v_a)
mp_type = _get_mp_type(f"../configs/IM/grid_id{gid}/simbox.yaml")
plot_title += f"substrate = {mp_type}"
bounds = (xmin, xmax, vmin, vmax, nbins)
title = {"title": plot_title, "size": 20}

X, Y, init_pts = init_lattice(F, bounds)

fig = analysis.F_streamplot(
    F,
    bounds,
    stream_init_pts=init_pts,
    title=title,
    interp="bilinear",
    streamplot_kwargs={
        "integration_direction": "forward",
        "color": "black",
        "broken_streamlines": False,
        "density": 1,
        "linewidth": 0.5,
    },
    # save_path=save_path + "full_lattice.png",
)
plt.show()

2. Color each lattice site depending on where it ends

In [ ]:
colors = []

for xx, yy in init_pts:
    fig, ax = plt.subplots(1, 1)
    try:
        stream = ax.streamplot(
            X,
            Y,
            Y,
            F,
            linewidth=0.5,
            start_points=[[xx, yy]],
            integration_direction="forward",
            color="black",
            broken_streamlines=False,
            density=1,
        )
        streamlines = np.array(stream.lines.get_segments())
        plt.close(fig)

        if len(streamlines) == 0:
            colors.append("gainsboro")
            continue

        end_pt = streamlines[-1][-1]
        if end_pt[0] < 133:
            colors.append("red")
        elif end_pt[0] > 167:
            colors.append("blue")
        else:
            colors.append("orange")

    except ValueError:
        colors.append("gainsboro")
        plt.close(fig)
        continue

In [ ]:
import matplotlib.patches as mpatches

red_patch = mpatches.Patch(color="red", label="Left")
blue_patch = mpatches.Patch(color="blue", label="Right")
orange_patch = mpatches.Patch(color="orange", label="Middle")

legend = [red_patch, blue_patch, orange_patch]

plt.figure(figsize=(3, 3), dpi=300)
plt.vlines(
    x=133, ymin=vmin, ymax=vmax, linewidth=1, colors=["black"], linestyles=["dashed"]
)
plt.vlines(
    x=167, ymin=vmin, ymax=vmax, linewidth=1, colors=["black"], linestyles=["dashed"]
)


plt.legend(handles=legend, bbox_to_anchor=(1.01, 1))
plt.scatter(init_pts[:, 0], init_pts[:, 1], color=colors, s=5)
plt.xlabel(r"$x$ ($\mu$m)")
plt.ylabel(r"$v$ ($\mu$m/hr)")
plt.savefig(save_path + "lattice_colors.png")
plt.close()

3. View stream line bunches per each category of end point behavior

In [ ]:
red_indx = np.where(np.array(colors) == "red")[0]
blue_indx = np.where(np.array(colors) == "blue")[0]
orange_indx = np.where(np.array(colors) == "orange")[0]

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(3, 3), dpi=300)
stream_colors = {"red": "lightcoral", "blue": "lightsteelblue", "orange": "wheat"}

for indx in [red_indx, blue_indx, orange_indx]:
    if len(indx) == 0:
        continue

    starting_pts = init_pts[indx]
    c = colors[indx[0]]
    stream_c = stream_colors[c]

    for xx, yy in starting_pts:
        stream = ax.streamplot(
            X,
            Y,
            Y,
            F,
            linewidth=1,
            start_points=[[xx, yy]],
            integration_direction="forward",
            color=stream_c,
            broken_streamlines=False,
            density=1,
        )
        streamlines = np.array(stream.lines.get_segments())

        if len(streamlines) == 0:
            continue

        start_pt = streamlines[0][0]
        end_pt = streamlines[-1][-1]
        ax.scatter(*end_pt, s=50, color=c, marker="*", zorder=5, label="end")
        ax.set_xlim(bounds[0:2])
        ax.set_ylim(bounds[2:4])
        ax.set_xlabel(r"$x$ ($\mu$m)")
        ax.set_ylabel(r"$v$ ($\mu$m/hr)")
        # ax.legend(bbox_to_anchor=(1.7, 1))
plt.savefig(save_path + f"lattice_segments.png")
plt.close()
# plt.show()